## Model Development

Based on the previous work in the other notebooks, we managed to gain some initial insight for what might be contributing features when it comes to determining an online users MBTI personality factors based on what they posted, such as the overall sentiment of their posts, length of their posts, noun and verb frequency of their posts, etc. Here we try to develop a model with more emphasis on performance.

### SMOTE + Word2Vec + Logistic Regression

In the paper *Ryan, G.; Katarina, P.; Suhartono, D. MBTI Personality Prediction Using Machine Learning and SMOTE for Balancing Data Based on Statement Sentences. Information 2023, 14, 217*, a combination of SMOTE oversampling, Word2Vec word embeddings and Logistic Regression was used to achieve an average F1 - Score of 0.8337 across the four dimensions of MBTI personalities, namely:
- Extraversion vs Introversion
- Sensing vs Intuition
- Thinking vs Feeling
- Judgment vs Perception

In this section we will attempt to implement this model from the paper using their methodology.

##### Preprocess Dataset

In [4]:
import pandas as pd
import numpy as np
import os
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

In [5]:
CSV_DATA_PATH = os.path.join("data", "mbti_2.csv")

def load_csv_data(csv_file_path):
    """ Load data from a given csv file into a pandas DataFrame object.

    Args:
    - csv_file_path (str) - The file path of the csv file containing the desired data to load into a pandas DataFrame object.

    Returns:
    - data (pandas.DataFrame) - The loaded data as a pandas DataFrame object.
    """
    assert csv_file_path.endswith(".csv")

    return pd.read_csv(csv_file_path)

def lemmatize_words(text):
    """ Lemmatize the input string of text.

    Args:
    - text (str) - The input string of text to lemmatize.

    Returns:
    - lemmatized (str) - The lemmatized version of the string of text that was given to the function.
    """
    lemmatizer = WordNetLemmatizer()

    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized = " ".join(words)

    return lemmatized

def preprocess_df(df):
    """ Helper function for preprocessing the input pandas dataframe by helping clean up input user posts.

    Args:
    - df (pd.DataFrame) - The input dataframe object containing the user posts to clean up.

    Returns:
    - df (pd.DataFrame) - The given dataframe object after cleaning has been applied to the user posts data.
    """   

    # Keep the End Of Sentence characters
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'\.', ' EOSTokenDot ', str(x) + " "))
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'\?', ' EOSTokenQuest ', str(x) + " "))
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'!', ' EOSTokenExs ', str(x) + " "))
    
    # Strip Punctation
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'[\.+]', ".", str(x)))

    # Remove multiple fullstops
    # df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'[^\w\s]','', str(x)))

    # Remove Non-words
    # df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','', str(x)))

    # Convert posts to lowercase
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: str(x).lower())

    # Remove multiple letter repeating words
    # df["posts_no_url"] = df["posts_no_url"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','', str(x)))
    
    # Strip trailing whitespaces
    df["posts_no_url"] = df["posts_no_url"].apply(lambda x: str(x).strip())

    # Remove rows with no text
    df["posts_no_url"] = df["posts_no_url"].replace('', np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace('nan', np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace("'", np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace("''", np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace('"', np.nan)
    df["posts_no_url"] = df["posts_no_url"].replace('""', np.nan)
    df.dropna(subset=["posts_no_url"], inplace=True)

    # Lemmatize posts
    df["posts_no_url"] = df["posts_no_url"].apply(lemmatize_words)

    # Generate word tokens
    df["posts_no_url_tokens"] = df["posts_no_url"].apply(word_tokenize)

    return df[["e_vs_i", "s_vs_n", "t_vs_f", "j_vs_p", "type", "posts", "posts_no_url", "posts_no_url_tokens"]]

def binarize_targets(df):
    """ Apply 0/1 labels to the input classes in the df.

    Args:
    - df (pd.DataFrame) - The input dataframe object containing the classes to numerically label.

    Returns:
    - df (pd.DataFrame) - The given dataframe object after numerically labelling classes.
    """
    binary_map = {
        'I': 0, 
        'E': 1, 
        'N': 0, 
        'S': 1, 
        'F': 0, 
        'T': 1, 
        'J': 0, 
        'P': 1
    }

    df["EI"] = df["e_vs_i"].apply(lambda x: binary_map[str(x)])
    df["SN"] = df["s_vs_n"].apply(lambda x: binary_map[str(x)])
    df["TF"] = df["t_vs_f"].apply(lambda x: binary_map[str(x)])
    df["JP"] = df["j_vs_p"].apply(lambda x: binary_map[str(x)])

    df["target_vec"] = df.apply(lambda x: [
        x["EI"],
        x["SN"],
        x["TF"],
        x["JP"]
    ], axis=1)

    return df

def preprocess_dataset(csv_file_path = CSV_DATA_PATH):
    """ Load and preprocess data to be used for model development.

    Args:
    - csv_file_path (str) - The file path of the csv file containing the desired data to load into a pandas DataFrame object.

    Returns:
    - numpy arrays of training and test split data.
    """

    # Apply preprocessing to input user posts
    df = load_csv_data(csv_file_path)
    df = preprocess_df(df)

    # Preprocess target labels into a binary vector
    df = binarize_targets(df)

    return df

In [6]:
df = preprocess_dataset()
df.head()

,e_vs_i,s_vs_n,t_vs_f,j_vs_p,type,posts,posts_no_url,posts_no_url_tokens,EI,SN,TF,JP,target_vec
2,I,N,F,J,INFJ,enfp and intj moments https://www.youtube.com...,enfp and intj moment sportscenter not top ten ...,"[enfp, and, intj, moment, sportscenter, not, t...",0,0,0,0,"[0, 0, 0, 0]"
3,I,N,F,J,INFJ,What has been the most life-changing experienc...,what ha been the most life-changing experience...,"[what, ha, been, the, most, life-changing, exp...",0,0,0,0,"[0, 0, 0, 0]"
4,I,N,F,J,INFJ,http://www.youtube.com/watch?v=vXZeYwwRDw8 h...,on repeat for most of today eostokendot,"[on, repeat, for, most, of, today, eostokendot]",0,0,0,0,"[0, 0, 0, 0]"
5,I,N,F,J,INFJ,May the PerC Experience immerse you.,may the perc experience immerse you eostokendot,"[may, the, perc, experience, immerse, you, eos...",0,0,0,0,"[0, 0, 0, 0]"
6,I,N,F,J,INFJ,The last thing my INFJ friend posted on his fa...,the last thing my infj friend posted on his fa...,"[the, last, thing, my, infj, friend, posted, o...",0,0,0,0,"[0, 0, 0, 0]"


The following preprocessing steps were applied to the dataset:
- Converting letters to lowercase
- Removing links
- Removing punctuations
- Removing stopwords

Lematization was also applied after the above steps have been conducted, and the resulting text for each post tokenized.

##### Word Embeddings and Text Vectorization